# <b>Mục tiêu</b>
- Thu thập dữ liệu về thời tiết tại TP HCM theo từng ngày từ năm 2010 đến năm 2021.

- Các trường dữ liệu sẽ thu thập bao gồm:
    + Weather Type: Loại thời tiết
    + Highest Temperature: Nhiệt độ cao nhất trong ngày ($^\circ C$)
    + Lowest Temperature: Nhiệt độ thấp nhất trong ngày ($^\circ C$)
    + Wind Speed: Tốc độ gió (km/h)
    + Rain: Lượng mưa (mm)
    + Humidity: Độ ẩm (%)
    + Cloud: Độ che phủ của mây (%)
    + Pressure: Áp suất không khí (mb)

# <b>Import</b>

In [1]:
import pandas as pd
import numpy as np
import re
import csv

In [2]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup

In [4]:
%pip install Selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install webdriver-manager

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# <b>Thu thập dữ liệu</b>

In [7]:
url = 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx'
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
months = 12
year = 2010
weather_info_list = []              # List chứa dữ liệu thời tiết TP HCM 2010 - 2021

In [8]:
# Thiết lập ChromeDriver
service = ChromeService(executable_path=ChromeDriverManager().install())
options = Options()
options.add_argument('--headless')                              # Ẩn màn hình Chrome, chỉ chạy dưới nền
options.add_argument('--window-size=1920, 1200')
driver = webdriver.Chrome(service= service, options= options)    # Khởi tạo Driver

driver.get(url)

for month in range(months):
    for date in range(days[month]):
        # Nhập ngày cần tìm thông tin thời tiết
        search_date = driver.find_element(by= By.XPATH, value= '//*[@id="ctl00_MainContentHolder_txtPastDate"]')
        driver.execute_script(f'arguments[0].setAttribute("value", "{year}-{month + 1:02}-{date + 1:02}");', search_date)      # Eg: 2010-02-01 

        # Nhấn nút 'Get Weather' để hiện thị dữ liệu thời tiết
        get_weather_button = driver.find_element(by= By.XPATH, value= '//*[@id="ctl00_MainContentHolder_butShowPastWeather"]')
        driver.execute_script('arguments[0].click();', get_weather_button)

        try:
            # Chờ thao tác tìm kiếm kết thúc mới tiến hành thu thập dữ liệu
            weather_info = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]'))
            )

            for i in range(2, 14):
                # Năm
                year_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}1]')

                # Loại thời tiết - Weather Type
                weather_type_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}2]/img')

                # Nhiệt độ cao nhất - Highest Temperature
                highest_temp_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}3]')

                # Nhiệt độ thấp nhất - Lowest Temperature
                lowest_temp_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}4]')

                # Tốc độ gió - Wind Speed
                wind_speed_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}5]')

                # Lượng mưa - Rain
                rain_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}6]')
                
                # Độ ẩm - Humidity
                humidity_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}7]')

                # Độ che phủ của mây - Cloud
                cloud_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}8]')

                # Áp suất không khí - Pressure
                pressure_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i}9]')
                
                # Lưu dữ liệu vào dictionary
                weather_in_date = {
                    'Date': f'{int(year_info.text)}-{month + 1:02}-{date + 1:02}',
                    'Weather Type': weather_type_info.get_attribute("title"),
                    'Highest Temperature': re.findall(r'\d+', highest_temp_info.text)[0],
                    'Lowest Temperature': re.findall(r'\d+', lowest_temp_info.text)[0],
                    'Wind Speed': re.findall(r'\d+', wind_speed_info.text)[0],
                    'Rain': re.findall(r'\d+', rain_info.text)[0],
                    'Humidity': re.findall(r'\d+', humidity_info.text)[0],
                    'Cloud': re.findall(r'\d+', cloud_info.text)[0],
                    'Pressure': re.findall(r'\d+', pressure_info.text)[0],
                }
                
                # Thêm thông tin lấy được vào list
                weather_info_list.append(weather_in_date)

        except:
            print('An error occurred! Close browser!')
            driver.quit()
            exit()

driver.quit()

# <b>Lưu dữ liệu</b>

In [9]:
header = list(weather_info_list[0].keys()) 
weather_info_list.sort(key=lambda item:item['Date'])

try:
   with open('weather_info.csv',mode='w',encoding='utf8',newline='') as output_to_csv:
       dict_csv_writer = csv.DictWriter(output_to_csv, fieldnames=header,dialect='excel')
       dict_csv_writer.writeheader()
       dict_csv_writer.writerows(weather_info_list)
   print('\nData exported to csv succesfully!')
except IOError as io:
    print('\n',io)


Data exported to csv succesfully!
